## Setup

In [1]:
from entsoe import EntsoePandasClient
import os
from dotenv import load_dotenv
from data_utils import query_and_save

In [2]:
load_dotenv()  # load env file
entsoe_key = os.getenv("ENTSOE_API_KEY")  # Get API keys from .env file

# Instantiate the ENTSOE Client
client = EntsoePandasClient(api_key=entsoe_key)

## Define Parameters

In [3]:
# Define the overall time range
start_year = 2019
end_year = 2024

# Define the countries, CORE CCR countries are: AT BE HR CZ FR DE HU LU NL PL RO SK SI   
countries = ["AT", "BE", "HR", "CZ", "FR", "DE_LU", "HU","NL", "PL", "RO", "SK", "SI", "PL", "CH"]

## Fetch Data
To fetch data from the ENTSOE API, simply run each cell. If there are connection errors, simply run the cell again. Re-running cells will only overwrite data if it's incomplete, so simply rerunning the cells will also update the data to the current date minus 2 days (to allow for delays with uploading the ENTSOE data)

### Day-ahead Prices

In [5]:
query_and_save(
    query_func=client.query_day_ahead_prices,
    filename_template="day_ahead_prices_{}.parquet",
    countries=countries, start_year=start_year, end_year=end_year)

Processing query_day_ahead_prices:   0%|          | 0/84 [00:00<?, ?it/s]


 Summary:

Successful queries: 84 / 84 country-years


### Load Forecast

In [6]:
query_and_save(
    query_func=client.query_load_forecast,
    filename_template="load_forecast_{}.parquet",
    countries=countries, start_year=start_year, end_year=end_year)

Processing query_load_forecast:   0%|          | 0/84 [00:00<?, ?it/s]


 Summary:

Successful queries: 84 / 84 country-years


### Generation Forecast

In [7]:
query_and_save(
    query_func=client.query_generation_forecast,
    filename_template="generation_forecast_{}.parquet",
    countries=countries, start_year=start_year, end_year=end_year)

Processing query_generation_forecast:   0%|          | 0/84 [00:00<?, ?it/s]


 Summary:

Successful queries: 83 / 84 country-years

No matching data for: HR_2019


### Wind and Solar Forecast

In [8]:
query_and_save(
    query_func=client.query_wind_and_solar_forecast,
    filename_template="wind_and_solar_forecast_{}.parquet",
    countries=countries, start_year=start_year, end_year=end_year)

Processing query_wind_and_solar_forecast:   0%|          | 0/84 [00:00<?, ?it/s]


 Summary:
FR_2019: 72 NaNs out of 17430 datapoints
FR_2020: 49 NaNs out of 17286 datapoints
FR_2021: 74 NaNs out of 17382 datapoints
FR_2022: 39 NaNs out of 17456 datapoints
FR_2023: 5208 NaNs out of 26139 datapoints
DE_LU_2023: 8 NaNs out of 104835 datapoints
HU_2019: 96 NaNs out of 69890 datapoints
PL_2020: 2398 NaNs out of 17522 datapoints

Successful queries: 83 / 84 country-years

No matching data for: HR_2019


### Installed Generation Capacity

In [7]:
query_and_save(
    query_func=client.query_installed_generation_capacity,
    filename_template="installed_generation_capacity_{}.parquet",
    countries=countries, start_year=start_year, end_year=end_year)

Processing query_installed_generation_capacity:   0%|          | 0/84 [00:00<?, ?it/s]


 Summary:

Successful queries: 81 / 84 country-years

No matching data for: SK_2022, SK_2023, SK_2024
